In [ ]:
library(bupaverse)
library(processanimateR)
library(tidyverse)
library(bupaR)
library(bupaverse)
library(processanimateR)
library(tidyverse)
library(bupaR)
library(lubridate)
library(plotly)

In [ ]:
#Se toma la base de datos con casos de córdoba
logs_causas_judiciales <- read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v2.csv.gz', delim=",")
options(repr.matrix.max.cols=21,repr.matrix.max.rows=500)

In [ ]:
#creamos la columna case_id
logs_causas_judiciales$case_id = paste(logs_causas_judiciales$causa_id, logs_causas_judiciales$unidad_id, sep="-")

In [ ]:
logs_causas_judiciales

In [ ]:
#Se quiere eliminar duplicidad de la actividad Inicio Causa

#1)Creamos un dataset sin la actividad
logs_causas_judiciales %>%
    #filter(causa_id== 186173) %>%
    filter(acto_procesal_descripcion != 'Inicio Causa') ->df_logs_causas_judiciales  #
df_logs_causas_judiciales


In [ ]:
#2)se hacen conversiones a algunas columnas
df_logs_causas_judiciales$objeto_litigio_id = as.numeric(as.character(df_logs_causas_judiciales$objeto_litigio_id)) 
df_logs_causas_judiciales$unidad_id = as.numeric(as.character(df_logs_causas_judiciales$unidad_id)) 
df_logs_causas_judiciales$materia_estadistica <- ifelse(df_logs_causas_judiciales$materia_estadistica == "Civil patrimonial y comercial", 
                      "Civil patrimonial y Comercial", 
                      df_logs_causas_judiciales$materia_estadistica)
df_logs_causas_judiciales$objeto_litigio_estadistico = toupper(df_logs_causas_judiciales$objeto_litigio_estadistico)

In [ ]:
#validamos conversiones
print(sapply(df_logs_causas_judiciales, class))

In [ ]:
#3)creamos un dataset temporal donde sumamos la actividad Inicio Causa, una actividad por caso
df_logs_causas_judiciales %>% 
    select(provincia_id, provincia_nombre,causa_id
           #,materia_id,materia_descripcion
           #  ,materia_estadistica
           #,circunscripcion_id,circunscripcion_descripcion
            #,unidad_id,unidad_descripcion
            ,causa_fecha_inicio
            #,objeto_litigio_id,objeto_litigio_descripcion,objeto_litigio_estadistico
           #,case_id
          ) %>%
    distinct()%>%
        filter(!is.na(causa_fecha_inicio))      %>%
mutate( acto_procesal_descripcion="Inicio Causa"
       ,acto_procesal_fecha=causa_fecha_inicio
       ,circunscripcion_id= NA,circunscripcion_descripcion=NA
       ,unidad_id=NA,unidad_descripcion=NA
       ,objeto_litigio_id=NA,objeto_litigio_descripcion=NA,objeto_litigio_estadistico=NA
       ,materia_id=NA,materia_descripcion=NA,materia_estadistica=NA
       ,estado_procesal='sd'
       ,fecha_envio= NA  
       ,case_id=NA
      ) %>%
    select(provincia_id, provincia_nombre,causa_id,materia_id,materia_descripcion
             ,materia_estadistica,circunscripcion_id,circunscripcion_descripcion
            ,unidad_id,unidad_descripcion,causa_fecha_inicio
            ,objeto_litigio_id,objeto_litigio_descripcion,objeto_litigio_estadistico,acto_procesal_descripcion
            ,acto_procesal_fecha,estado_procesal,fecha_envio,case_id
          )->tmp

In [ ]:
#tmp%>% filter(causa_id=='6271838')

In [ ]:
#4) unimos ambos datasets, el que no tiene la actividad Inicio Causa y el temporal creado en el paso 3
df_logs_causas_judiciales_output <- rbind(df_logs_causas_judiciales,tmp)


In [ ]:
df_logs_causas_judiciales_output #%>% filter(causa_id=='6271838' & acto_procesal_descripcion=="Inicio Causa") #filter(causa_id=='6271838')

In [ ]:
#5) Validamos que no hay más actividades Inicio Causa duplicadas
df_logs_causas_judiciales_output%>%
filter(acto_procesal_descripcion=="Inicio Causa") %>%
 group_by(causa_id,acto_procesal_descripcion)%>%
    #summarise(conteo = n()) -> validacion
    count() -> validacion
validacion %>% filter(n>1)

In [ ]:
#6) creamos un nuevo dataset con el output previo
#write.csv(logs_causas_judiciales,file='./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v2_1.csv.gz',row.names=FALSE)
write.csv(df_logs_causas_judiciales_output,file='./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v2_2.csv.gz',row.names=FALSE)




In [ ]:
#7) al dataset creado previamente lo cargamos nuevamente
df_causas_judiciales <- df_logs_causas_judiciales_output
    #read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v2_1.csv.gz', delim=",")
    #read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v2_2.csv.gz', delim=",")

In [ ]:
#8) cargamos un segundo dataset donde está la revisión de los actos procesales, que indica cómo convertir cada acto en una actividad
df_Revision_ActosProcesales <- 
    read_delim(file = './base_datos/UNZIP_Files/Revision_ActosProcesales.csv', delim=",")

In [ ]:
# Limpiar los datos (convertir a minúsculas y eliminar espacios)
df_causas_judiciales$Col1_clean <- tolower(trimws(df_causas_judiciales$acto_procesal_descripcion))  # Limpiar Col1 de df1
df_Revision_ActosProcesales$ColA_clean <- tolower(trimws(df_Revision_ActosProcesales$acto_procesal))  # Limpiar ColA de df2

In [ ]:
#9) hacemos un join entre ambos datasets para sumar las columnas del segundo dataset
df_causas_judiciales_rev <- left_join(df_causas_judiciales, df_Revision_ActosProcesales, by = c("Col1_clean" = "ColA_clean"))

In [ ]:
#10) guardamos el output previo en un nuevo dataset
#write.csv(df_causas_judiciales_rev, gzfile("./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v3_2.csv.gz"), row.names = FALSE)
write.csv(df_causas_judiciales_rev, gzfile("./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v3_3.csv.gz"), row.names = FALSE)



In [ ]:
#11) cargamos nuevamente el dataset y filtramos casos del fuero civil para armar la base de datos final q alimentara a la app
df_causas_judiciales <- df_causas_judiciales_rev
    #read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v3_2.csv.gz', delim=",")
    #read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v3_3.csv.gz', delim=",")
#df_causas_judiciales$acto_procesal_fecha <- paste0(df_causas_judiciales$acto_procesal_fecha, ' 00:00:00')
df_causas_judiciales %>%
    filter( materia_estadistica == 'Civil patrimonial y comercial' | materia_estadistica == 'Civil patrimonial y Comercial') -> df_causas_judiciales_v4

In [ ]:
#12) se crea un nuevo data set
#write.csv(df_causas_judiciales_v4,file='./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v4_1.csv.gz',row.names=FALSE)
write.csv(df_causas_judiciales_v4,file='./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v4_1_1.csv.gz',row.names=FALSE)



In [ ]:
# 12) se carga el ultimo data set creado y se suma la columna de throughput time
df_causas_judiciales <- df_causas_judiciales_v4
    #read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v4_1.csv.gz', delim=",")
    #read_delim(file = './base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v4_1_1.csv.gz', delim=",")
colnames(df_causas_judiciales)

In [ ]:
#transformamos el dataset en un eventlog
df_causas_judiciales$acto_procesal_fecha <- paste0(df_causas_judiciales$acto_procesal_fecha, ' 00:00:00')
df_causas_judiciales %>%
    # rename timestamp variables appropriately
    dplyr::rename(start = acto_procesal_fecha#, 
           #complete = acto_procesal_fecha
                    ) %>%
    # convert timestamps to 
    convert_timestamps(columns = "start", format = ymd_hms) %>%
    activitylog(case_id = "causa_id",
                activity_id = "actividad",
                timestamps = "start",
                resource_id="unidad_id") -> lg_logs_causas_judiciales
colnames(lg_logs_causas_judiciales)

In [ ]:
#suamos la col throughput time
lg_logs_causas_judiciales <- data.frame(lg_logs_causas_judiciales  
                                        %>% throughput_time("case",units="days") 
                                        %>%  augment(lg_logs_causas_judiciales))
#lg_logs_causas_judiciales$throughput_time <- as.numeric(lg_logs_causas_judiciales$throughput_time)
colnames(lg_logs_causas_judiciales)

In [ ]:
#validamos conversion tipo datos
#print(sapply(lg_logs_causas_judiciales, class))

In [ ]:
lg_logs_causas_judiciales %>% filter(causa_id=='6271838')   

In [ ]:
#seleccionamos nuevamente las columnas q necesitamos para crear el dataset final
nvo_df_causas_judiciales <- data.frame(lg_logs_causas_judiciales %>%
    select(provincia_id, provincia_nombre,causa_id,materia_id,materia_descripcion
             ,materia_estadistica,circunscripcion_id,circunscripcion_descripcion
            ,unidad_id,unidad_descripcion,causa_fecha_inicio
            ,objeto_litigio_id,objeto_litigio_descripcion,objeto_litigio_estadistico,acto_procesal_descripcion
            ,start,estado_procesal,fecha_envio,Col1_clean,acto_procesal,actividad,throughput_time,complete,'.order'
          ))
colnames(nvo_df_causas_judiciales)

In [ ]:
nvo_df_causas_judiciales <- nvo_df_causas_judiciales%>%
    select(provincia_id, provincia_nombre,causa_id,materia_id,materia_descripcion
             ,materia_estadistica,circunscripcion_id,circunscripcion_descripcion
            ,unidad_id,unidad_descripcion,causa_fecha_inicio
            ,objeto_litigio_id,objeto_litigio_descripcion,objeto_litigio_estadistico,acto_procesal_descripcion
            ,'acto_procesal_fecha'=start,estado_procesal,fecha_envio,Col1_clean,acto_procesal,actividad,throughput_time
          )
colnames(nvo_df_causas_judiciales)

In [ ]:
#12) se crea la BD final
#write.csv(nvo_df_causas_judiciales,file='./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v4_2.csv.gz',row.names=FALSE)
write.csv(nvo_df_causas_judiciales,file='./base_datos/UNZIP_Files/combined_tabla_actos_procesales_no_penales_cordoba_v4_2_1.csv.gz',row.names=FALSE)


